In [ ]:
import pandas as pd
import cv2
import numpy as np
import glob
import matplotlib.pyplot as plt
import face_recognition

In [ ]:
dataset = 'dataset/datasets/'
images = glob.glob(dataset+"*/*")
np.random.shuffle(images)

In [7]:
data_face = []
for i in images:
    image = face_recognition.load_image_file(i)
    face_locations = face_recognition.face_locations(image)
    try:
        (top, right, bottom, left) = face_locations[0]
        img_ = image[top:bottom, left:right, :]
        img_ = cv2.resize(img_, (100, 100))
        data_face.append(img_)
    except:
        print('error', i)

error dataset/datasets/toufan/6.jpg
error dataset/datasets/zul/18.jpg
error dataset/datasets/surahmin/15.png
error dataset/datasets/cezar/18.jpg
error dataset/datasets/cezar/13.jpg
error dataset/datasets/cezar/21.jpg
error dataset/datasets/cezar/14.jpg
error dataset/datasets/cezar/17.jpg
error dataset/datasets/cezar/5.jpg


In [ ]:
import pickle
f = open('face_images.pickle', 'wb')
f.write(pickle.dumps(data_face))
f.close()

In [ ]:
face_flip = []
for j in data_face:
    for k in range(2):
        flip = cv2.flip(j, k)
        face_flip.append(flip)

In [ ]:
final_aug_img = []
for a in face_flip:
    (h, w) = a.shape[:2]
    center = (h // 2, w // 2)
    for deg in range(len(face_flip)):
        M = cv2.getRotationMatrix2D(center, deg, 1.0)
        rotated = cv2.warpAffine(a, M, (w, h))
        final_aug_img.append(rotated)

In [ ]:
np.shape(final_aug_img)

In [ ]:
cv2.imshow('img',data_face[100])
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
# aug_img = []
# for j in images[:]:
#     gbr = cv2.imread(j)
#     gbr = cv2.resize(gbr, (32,32))
#     (h, w) = gbr.shape[:2]
#     center = (h // 2, w // 2)
#     for jj in range(len(images)):
#         M = cv2.getRotationMatrix2D(center, jj, 1.0)
#         rotated = cv2.warpAffine(gbr, M, (w, h))
#         aug_img.append(rotated)

In [ ]:
np.shape(fa)

In [ ]:
# cv2.imshow('img', img_)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [ ]:
# kita load datanya ke memory
image_data = []
label_data = []
for i, ii in zip(aug_img, images):
    image_data.append(i)
    lbl = ii.split('/')[-2]
    label_data.append(lbl)

In [ ]:
x = np.asarray(image_data)
y = np.asarray(label_data)

In [ ]:
x = np.float32(x)
x = x / 255

In [ ]:
from sklearn.preprocessing import LabelBinarizer
lb = LabelBinarizer()
y_en = lb.fit_transform(y)

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y_en, test_size=0.25, random_state=42)
# cek apakah sudah benar atau belum pemecaannya
print('x_train : ',x_train.shape)
print('x_test  : ', x_test.shape)
print('y_train : ', y_train.shape)
print('y_test  : ', y_test.shape)

In [ ]:
# keras model
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense
from keras.models import Input, Model, Sequential
# mulai membuat arsitektur NN dengan tipe fully connected layer
from keras.layers import BatchNormalization, Dropout
input_layer    = Input(shape=(x_train.shape[1:]))

batch = BatchNormalization(input_shape=x_train.shape[1:])(input_layer)
conv = Conv2D(16, kernel_size=(5,5), padding='SAME', activation='relu', name='conv_layer_1')(batch)
pool = MaxPool2D(pool_size=(4,4), name='maxpooling_1')(conv)
drop = Dropout(0.25)(pool)

batch1 = BatchNormalization(input_shape=x_train.shape[1:])(drop)
conv1 = Conv2D(16, kernel_size=(5,5), padding='SAME', activation='relu', name='conv_layer_2')(batch1)
pool1 = MaxPool2D(pool_size=(4,4), name='maxpooing_2')(conv1)
drop1 = Dropout(0.25)(pool1)

flatten_layer = Flatten(name='flatten_layer')(drop1)
full_con_layer = Dense(512, activation='relu', name='fully_connected_layer')(flatten_layer)
full_con_layer1 = Dense(128, activation='relu', name='fully_connected_layer_1')(full_con_layer)
drop3 = Dropout(0.5)(full_con_layer1)
output_layer = Dense(6, activation='softmax', name='output_layer')(drop3)
model = Model(inputs=input_layer, outputs=output_layer)
# model.summary()

In [ ]:
import keras
from keras.callbacks import ReduceLROnPlateau
Tensorboard = keras.callbacks.TensorBoard(log_dir='./keras-logs-2', histogram_freq=0,  write_graph=True, write_images=False)
# Save the checkpoint in the /output folder
# filepath = "/keras-ckpt/mnist-cnn-best.hdf5"

# # Keep only a single checkpoint, the best over test accuracy.
# checkpoint = keras.callbacks.ModelCheckpoint(filepath,
#                             monitor='val_acc',
#                             verbose=1,
#                             save_best_only=True,
#                             mode='max')

from keras.optimizers import SGD
opt = SGD(lr=2)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=5, min_lr=0.001)

model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

In [ ]:
history = model.fit(x_train, y_train, 
                    batch_size=32, epochs=890,
                   validation_data=[x_test, y_test], 
                    callbacks=[Tensorboard, reduce_lr])
model.save('roni_1.h5')

In [ ]:
# nilai-nilai loss-nya
loss = history.history['loss']

# nilai-nilai val_loss-nya
val_loss = history.history['val_loss']

# nilai-nilai akurasi-nya
acc = history.history['acc']

# nilai-nilai val_acc-nya
val_acc = history.history['val_acc']

# menentukan sumbu x nya dari 0 sampai dengan banyaknya jumlah nilai loss/acc
sumbu_x = np.arange(0, len(loss))

# memasukan library plotting (matplotlib)
import matplotlib.pyplot as plt

# grafik plot untuk melihat loss
plt.figure(figsize=(15,3))
plt.subplot(1,2,1)
plt.plot(sumbu_x, loss, label='loss')
plt.plot(sumbu_x, val_loss, label='val_loss')
plt.legend()

# grafik plot untuk melihat akurasinya
plt.subplot(1,2,2)
plt.plot(sumbu_x, acc, label='acc')
plt.plot(sumbu_x, val_acc, label='val_acc')
plt.legend()
plt.show()

In [ ]:
model.evaluate(x_train, y_train)

In [ ]:
model.evaluate(x_test, y_test)